# TO DO:
- connect wedwards code for separating voyages
- Mess with classifier parameters (once working)
- test classifier for 2 validation sets


In [1]:
import collections

import numpy as np
import pandas as pd
import exploringShipLogbooks.wordCount as wc

from fuzzywuzzy import fuzz
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB

from exploringShipLogbooks.basic_utils import clean_data
from exploringShipLogbooks.basic_utils import encode_data_df
from exploringShipLogbooks.basic_utils import extract_logbook_data
from exploringShipLogbooks.basic_utils import isolate_columns
from exploringShipLogbooks.basic_utils import isolate_training_data

from exploringShipLogbooks.config import *

# Load and clean data
### Load CLIWOC ship logs

In [2]:
# extract data from zip file
cliwoc_data = extract_logbook_data('CLIWOC15.csv')
#cliwoc_data = cliwoc_data.loc[50000:100000]

/Applications/miniconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (5,6,7,8,11,13,18,19,23,24,25,26,28,29,30,34,35,38,43,44,46,73,77,81,82,84,85,87,88,94,96,97,98,99,111,114,116,119,120,122,124,125,127,129,131,133,135,137,140) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
label_encoding = preprocessing.LabelEncoder().fit(cliwoc_data['LogbookIdent']).classes_
cliwoc_data['LogbookIdent'] = preprocessing.LabelEncoder().fit_transform(cliwoc_data['LogbookIdent'])

### Find definite slave data in CLIWOC data set
- These logs will be used to test the classifier

In [4]:
# extract logs that mention slaves
slave_mask = wc.count_key_words(cliwoc_data, text_columns, slave_words)
print('Found ', len(slave_mask[slave_mask==True]), ' logs that mention slaves')

Found  464  logs that mention slaves


### Clean CLIWOC data

In [5]:
# find indices of ship names that are "non-slave" ships before dropping ship name column
cliwoc_data['ShipName'] = cliwoc_data['ShipName'].astype(str).map(lambda x: x.lower().rstrip())
non_slave_log_locations = cliwoc_data['ShipName'].isin(non_slave_ships)
print('Found ', len(non_slave_log_locations[non_slave_log_locations==True]), ' logs that are non-slave ships')

Found  4105  logs that are non-slave ships


In [6]:
cliwoc_data['slave_logs'] = np.zeros(len(cliwoc_data))
slave_log_locations = cliwoc_data['LogbookIdent'].isin(list(cliwoc_data['LogbookIdent']
                                                            [slave_mask].unique()))

- cliwoc_data (unclassified) = 0
- cliwoc_data (no slaves)    = 1
- cliwoc_data (slaves)       = 2
- slave_data                 = 3

In [7]:
cliwoc_data.loc[slave_log_locations,'slave_logs'] = 2
cliwoc_data.loc[non_slave_log_locations,'slave_logs'] = 1

In [8]:
cliwoc_data = cliwoc_data.sort_values('LogbookIdent', ascending=True)
cliwoc_data = cliwoc_data.drop_duplicates('LogbookIdent')
cliwoc_data = cliwoc_data.set_index('LogbookIdent')

In [9]:
# remove undesired columns
cliwoc_data = isolate_columns(cliwoc_data, desired_columns)

### Load Slave Voyages data

In [10]:
file_name = './exploringShipLogbooks/data/tastdb-exp-2010'
slave_voyage_logs = pd.read_pickle(file_name)

In [11]:
year_ind = ~(slave_voyage_logs['yeardep'].isnull())
slave_voyage_logs = slave_voyage_logs[year_ind]

In [12]:
cliwoc_ind = (slave_voyage_logs['yeardep']>cliwoc_data['Year'].min()) & (slave_voyage_logs['yeardep']<cliwoc_data['Year'].max())
slave_voyage_logs = slave_voyage_logs[cliwoc_ind]

### Clean Slave voyages data

In [13]:
from collections import OrderedDict

In [14]:
#slave_voyage_conversions=OrderedDict()
slave_voyage_conversions = OrderedDict([('portdep', 'VoyageFrom'), ('portret', 'VoyageTo'),
                            ('rig', 'ShipType'), ('national', 'Nationality'),
                            ('yeardep', 'Year')])
list(OrderedDict.fromkeys(slave_voyage_conversions))
list(slave_voyage_conversions.keys())

['portdep', 'portret', 'rig', 'national', 'yeardep']

In [16]:
slave_voyage_desired_cols = list(slave_voyage_conversions.keys())
slave_voyage_logs = isolate_columns(slave_voyage_logs, slave_voyage_desired_cols)

slave_voyage_logs.rename(columns=slave_voyage_conversions, inplace=True)
#slave_voyage_logs.columns = ['Nationality', 'ShipType', 'VoyageFrom', 'VoyageTo', 'Year']

In [17]:
slave_voyage_logs['slave_logs'] = 3
slave_voyage_indices = range(len(slave_voyage_logs)) + (cliwoc_data.tail(1).index[0]+1)
slave_voyage_logs = slave_voyage_logs.set_index(slave_voyage_indices)

In [18]:
cliwoc_data.tail(5)

,VoyageFrom,VoyageTo,ShipType,Nationality,Year,slave_logs
LogbookIdent,,,,,,
1646,Texel,Bengal,NaN,Dutch,1786,0
1647,Texel,Batavia,NaN,Dutch,1788,2
1648,Middelburg,Guinea,NaN,Dutch,1753,0
1649,Vlissingen,Guinea,Brik,Dutch,1788,0
1650,NaN,NaN,Fregat,Dutch,1762,0


In [19]:
slave_voyage_logs.head()

,Nationality,ShipType,VoyageFrom,VoyageTo,Year,slave_logs
1651,Portugal,Bergantim,Rio de Janeiro,Alicante,1816,3
1652,Spain,Bergantim,"Bahia, port unspecified",Alicante,1816,3
1653,Spain,Bergantim,"Bahia, port unspecified",Alicante,1816,3
1654,Spain,Bergantim,"Bahia, port unspecified",Alicante,1816,3
1655,Spain,Galera,Alicante,Alicante,1817,3


# Join data sets

In [20]:
#all_data = pd.concat([cliwoc_data, slave_voyage_logs])
all_data = cliwoc_data.append(slave_voyage_logs)
all_data = clean_data(all_data)

# cleanup (my computer won't run the code without this :( )
#del cliwoc_data, slave_voyage_logs

## Test of fuzzywuzzy method

In [ ]:
 df = pd.DataFrame( {'id':[1, 2, 3, 4, 5, 6], 'name':['dog', 'cat', 'mad cat', 'good dog', 'bad dog', 'chicken']})

In [ ]:
def func(name):
    matches = df.apply(lambda row: (fuzz.partial_ratio(row['name'], name) >= 85), axis=1)
    return [i for i, x in enumerate(matches) if x]

In [ ]:
df.apply(lambda row: func(row['name']), axis=1)

## Try fuzzywuzzy on subset of one of our columns

In [ ]:
def func(name, column_name):
    matches = all_data[0:10].apply(lambda row: (fuzz.partial_ratio(row[column_name], name) >= 85), axis=1)
    return [i for i, x in enumerate(matches) if x]

In [ ]:
column_name = 'Nationality'
all_data[0:10].apply(lambda row: func(row[column_name], column_name), axis=1)

# Encode data
- Must encode data before separating, otherwise values that do not occur in a subset will be encoded differently

In [21]:
all_data = encode_data_df(all_data)

## Remove nan columns from encoding (currently there are no empty cells)

In [22]:
all_data['no_data'] = all_data['nan'].apply(lambda x: x.any(), axis=1).astype(int)

In [23]:
all_data = all_data.drop('nan', axis=1)

In [24]:
all_data.head()

,argentina,brazil,british,danish,denmark,dutch,france,french,great britain,hamburg,...,willemsoord,williamsburg,woolwich,york factory,"york harbour, newfoundland",zeeland,zierikzee,Year,slave_logs,no_data
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1785,0,1
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1789,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1847,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1849,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1848,0,0


# Extract training data, and create list of classes

In [25]:
unclassified_logs = all_data[all_data['slave_logs']==0]

validation_set_1 = all_data[all_data['slave_logs']==2]

# reserve first 20% of slave_voyage_logs as validation set
validation_set_2_indices = range(slave_voyage_indices.min(),
                                 slave_voyage_indices.min() + round(len(slave_voyage_indices)*.2))
validation_set_2 = all_data.iloc[validation_set_2_indices]

training_logs_pos = all_data[all_data['slave_logs']==3]
training_logs_pos = training_logs_pos.drop(validation_set_2_indices)

# note! This relies on cliwoc data being first in all_data
# could make more robust later
# TODO: check that this works!! should have some in first 1000?
training_logs_neg = all_data[all_data['slave_logs']==1]

# cleanup
#del all_data

- left this code so we can check if there are any null values in each 
  dataframe

In [ ]:
def finding_null_values(df):
    return df.isnull().sum()[df.isnull().sum()>0]

In [26]:
# create list of classes for training data (0 is for non-slave, 1 is for slave)
# index matches training_data
classes = np.zeros(len(training_logs_neg))
classes = np.append(classes, np.ones(len(training_logs_pos)))

# joint training data
training_data = pd.concat([training_logs_neg, training_logs_pos], ignore_index = True)

# convert to numpy array
training_data = training_data.as_matrix()

# Fit training data to classifier
- **note!** first column of numpy array is index! do not include in classification!

In [27]:
classifier = MultinomialNB(alpha = 1.0, class_prior = None, fit_prior = True)
classifier.fit(training_data[::,1::], classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

# Test classifier
- check if slave logs from cliwoc data were classified correctly (want mostly classified as 1)
- compare first column with slave_index


In [31]:
print('Testing validation data from slave logs data set')
validation_test(classifier, validation_set_2, 1)

print('Testing validation data from cliwoc data set:')
validation_test(classifier, validation_set_1, 1)

Testing validation data from slave logs data set
Validation set was classified as a slave ship  100.0 % of the time
Testing validation data from cliwoc data set:
Validation set was classified as a slave ship  100.0 % of the time


In [29]:
def validation_test(classifier, validation_set, expected_class):
    """
    input classifer object, validation set (data frame), and expected class 
    of validation set (i.e. 1 or 0). Prints successful classification rate.
    """
    validation_set = validation_set.as_matrix()
    predictions = classifier.predict(validation_set[::,1::])
    
    counts = collections.Counter(predictions)
    percent_correct = (counts[expected_class]/(len(predictions))* 100)
                       
    print('Validation set was classified as a slave ship ',
          round(percent_correct,2), '% of the time')